In [1]:
import filereader
import numpy as np
import gurobipy as gp
import scipy

In [2]:
groceries_dict = filereader.read_grocery_info()

In [3]:
recipes_dict = filereader.read_recipes()

In [4]:
nutrition_needs_dict = filereader.read_nutrition()

In [6]:
# generate the matrices that we care about
# we first need to be certain of order of stuff, so let's generate these lists of keys
groceries_list = [key for key in groceries_dict.keys()]
recipes_list = [key for key in recipes_dict.keys()]
nutrients_list = [key for key in nutrition_needs_dict.keys()]

# # generate the N matrix (but this will actually be a dictionary)
# N = {}
# for recipe, ingredients in recipes_dict.items():
#     N[recipe]

N = [
    [
        sum([groceries_dict[ingredient]["nutrients"][nutrient]/groceries_dict[ingredient]["Serving Size (g)"]*recipes_dict[recipe][ingredient] for ingredient in recipes_dict[recipe]])
    for nutrient in nutrients_list]
    for recipe in recipes_list
]

# c = np.array([groceries_dict[ingredient]["Price (CAD)"] for ingredient in groceries_list])

In [26]:
# need to transpose the recipes dict
recipes_dict_transpose = {}

for ik in groceries_dict.keys():
    recipes_dict_transpose[ik] = {}

for rk, rv in recipes_dict.items():
    for ik, iv in rv.items():
        recipes_dict_transpose[ik][rk] = iv

# first entry is a list, with an entry for each ingredient, containing a list of the indices of all the recipes
# the second entry is a list of lists of corresponding masses
recipes_transpose_csr_ish = [[],[]]
for ik, iv in recipes_dict_transpose.items():
    recipes_transpose_csr_ish[0].append([recipes_list.index(recipe) for recipe in iv.keys()])
    recipes_transpose_csr_ish[1].append([mass for mass in iv.values()])


In [7]:
N_dict = {}
for rk, rv in recipes_dict.items():
    nutrients_dict = {}
    for nutrient in nutrition_needs_dict.keys():
        nutrients_dict[nutrient] = 0
    for gk, gv in rv.items():
        for nk, nv in groceries_dict[gk]["nutrients"].items():
            nutrients_dict[nk] += nv*gv/groceries_dict[gk]["Serving Size (g)"]
    N_dict[rk] = nutrients_dict


In [8]:
# [((k,j), N[k][j]) for k in range(len(nutrients_list)) for j in range(len(recipes_list))]

In [9]:
# N_ihategurobi = dict([((k,j), N[k][j]) for k in range(len(nutrients_list)) for j in range(len(recipes_list))])
# N_ihategurobi = {}
# for k in range(len(nutrients_list)):
#     for j in range(len(recipes_list)):
#         # print(f"{k},{j}")
#         # print(N[k][j])
#         N_ihategurobi[(k,j)] = N[j][k]
N_ihategurobi = []
for k in range(len(nutrients_list)):
    N_ihategurobi.append({})
    for j in range(len(recipes_list)):
        # print(f"{k},{j}")
        # print(N[k][j])
        N_ihategurobi[-1][(j,)] = N[j][k]

In [10]:
N_ihategurobi

[{(0,): 2035.92,
  (1,): 6157.55,
  (2,): 877.53,
  (3,): 1215.88,
  (4,): 1226.835,
  (5,): 2205.895,
  (6,): 4404.3,
  (7,): 2465.93,
  (8,): 1179.52,
  (9,): 2404.32,
  (10,): 1301.12,
  (11,): 2137.04,
  (12,): 1169.6100000000001,
  (13,): 2744.5099999999998,
  (14,): 1027.69,
  (15,): 1588.365,
  (16,): 4049.16,
  (17,): 169.1},
 {(0,): 170.974,
  (1,): 685.935,
  (2,): 49.18600000000001,
  (3,): 39.47,
  (4,): 38.61000000000001,
  (5,): 177.412,
  (6,): 116.938,
  (7,): 261.91200000000003,
  (8,): 75.842,
  (9,): 183.274,
  (10,): 76.304,
  (11,): 198.418,
  (12,): 11.765,
  (13,): 230.525,
  (14,): 70.764,
  (15,): 129.025,
  (16,): 144.10399999999998,
  (17,): 2.0900000000000003},
 {(0,): 541.46,
  (1,): 2059.15,
  (2,): 2149.79,
  (3,): 835.78,
  (4,): 140.32500000000002,
  (5,): 3862.51,
  (6,): 2083.8,
  (7,): 1288.03,
  (8,): 2094.21,
  (9,): 1140.06,
  (10,): 1070.67,
  (11,): 2373.58,
  (12,): 2416.46,
  (13,): 1363.78,
  (14,): 1546.34,
  (15,): 3554.775,
  (16,): 3224.5

In [11]:
len(recipes_list)

18

In [12]:
len(N)

18

In [13]:
# here, we define the cost, which is time-dependent
# for this reason, we need to be clear about the number of days we are considering

# the number of days we consider (one month)
t_max = 30

# the standard deviation for the random variation in price
price_sigma = 0.2

# costs for each item. Each is a list of 30 entries, one for each day
c_dict = {}
for gk, gv in groceries_dict.items():
    c_dict[gk] = gv["Price (CAD)"]*np.random.normal(loc=1, scale=price_sigma, size=t_max)

c_list = [gv["Price (CAD)"]*np.random.normal(loc=1, scale=price_sigma, size=t_max) for gk, gv in groceries_dict.items()]

# masses
m_dict = {}
for gk, gv in groceries_dict.items():
    m_dict[gk] = gv["Mass (g)"]

m_list = [gv["Mass (g)"] for gk, gv in groceries_dict.items()]

a_k = [nv for nk, nv in nutrition_needs_dict.items()]

In [14]:
model = gp.Model("New problem")

## seems that summation doesn't work unless we use the Gurobi addVars
# g_vars = {}
# u_vars = {}
# for gk in groceries_dict.keys():
#     u_vars[gk] = []
#     g_vars[gk] = model.addVars(t_max, vtype=gp.GRB.INTEGER, name=gk)
#     for t in range(t_max):
#         u_vars[gk].append(model.addVars(t+1, name=f"grams of {gk} used on day {t}"))


# r_vars = {}
# for rk in recipes_dict.keys():
#     r_vars[rk] = model.addVars(t_max, name=rk)

g_vars = model.addVars(len(groceries_list), t_max, vtype=gp.GRB.INTEGER, name="g[i][t]")
u_vars = model.addVars(len(groceries_list), t_max, t_max, name="u[i][tau][t]")
# r_vars = model.addVars(len(recipes_list), t_max, name="r[j][t]")
r_vars = model.addMVar(len(recipes_list), t_max, name="r[j][t]")

s = model.addVars(t_max, vtype=gp.GRB.BINARY, name="s")

model.setParam("LogToConsole", 1)  # Enable console logging
model.setParam("OutputFlag", 1)    # Enable all output

model.update()


Restricted license - for non-production use only - expires 2026-11-23
Set parameter LogToConsole to value 1
Set parameter OutputFlag to value 1


In [15]:
(np.array(a_k)*np.expand_dims(np.arange(t_max), axis=1)).shape

(30, 16)

In [18]:
np.array(a_k)

array([3.0e+03, 5.6e+01, 2.3e+03, 4.7e+03, 1.3e+03, 4.1e+02, 1.1e+01,
       1.1e+01, 9.0e+02, 1.2e+00, 1.3e+00, 1.6e+01, 1.3e+00, 2.4e+00,
       7.5e+01, 1.5e+01])

In [52]:
# %pdb
# # add the constraints
# # daily nutrient requirement
# # for nk, nv in N_dict[rk].items():
# for nutrient in nutrition_needs_dict.keys():
#     model.addConstrs(
#         (sum([r_vars[rk][t]*N_dict[rk][nutrient] for rk in r_vars.keys()])
#             >= nutrition_needs_dict[nutrient] for t in range(t_max)), name=nutrient)

# for t in range(t_max):
#     for k in range(len(a_k)):
#         model.addConstr(
#         sum(N[rk][k]*r_vars[rk][t] for rk in range(len(recipes_list)))
#         >= a_k[k]
#         )

# for t in range(t_max):
#     model.addMConstr(np.array(N).transpose(), r_vars, '>', np.array(a_k))
# for t in range(t_max):
    # model.addMConstr(np.array(N).transpose(), r_vars[:,t], '>', np.array(a_k)*np.expand_dims(np.arange(t_max), axis=1))
model.addMConstr(np.array(N).transpose(), r_vars, '>', np.array(a_k))


# model.addConstrs(
#     (
#         sum(N[rk][k]*r_vars[rk, t] for rk in range(len(recipes_list)))
#         >= a_k[k]
#         for k in range(len(a_k))
#         for t in range(t_max)
# ), name="nutritional needs met")

# model.addConstrs((
#     (
#         sum(r_vars[rk][t]*N[rk][k] for rk in range(len(recipes_list)))
#         >= a_k[k]
#         for k in range(len(a_k))
#     ) for t in range(t_max)
# ), name="nutritional needs met")


# model.addConstrs(
#     (sum(r_vars[rk, t] * N[rk][k] for rk in range(len(recipes_list))) >= a_k[k]
#      for k in range(len(a_k))
#      for t in range(t_max)),
#     name="nutritional_needs_met"
# )

# model.addConstrs(
#     {(k, t): sum(r_vars[rk, t] * N[rk][k] for rk in range(len(recipes_list))) >= a_k[k]
#      for k in range(len(a_k)) for t in range(t_max)},
#     name="nutritional_needs_met"
# )


# can only buy stuff if he pays for Instacart
# for grocery in groceries_dict.keys():
# model.addConstrs(((g_vars[gk][t] <= 999999999999999*s[t] for gk in groceries_dict.keys()) 
#                   for t in range(t_max)), 
#                   name="Buy only on shopping days")

model.addConstrs((g_vars[gk, t] <= 999999999999999*s[t] for gk in range(len(groceries_list))
                  for t in range(t_max)), 
                  name="Buy only on shopping days")

# need enough to cook with, and it can't expire
# for ik, iv in groceries_dict.items():
#     model.addConstrs(
#         (
#             recipes_dict[rk][ik]*r_vars[rk]
#         )
#     )

# for ik, iv in recipes_dict_transpose:
    # model.addConstrs(
    #     (
    #         (R_ij*r_vars[j][t] == (u_vars[ik][t][tau] for tau in range(max(0, t-groceries_dict[ik]["Shelf Life (Days)"]), t))
    #          for j, R_ij in iv.items())
    #     )
    # )
for t in range(t_max):
    model.addConstrs(
        (
            u_vars[i,tau,t] <= g_vars[i,tau]*m_list[i] - u_vars.sum(i,tau,'*') for i in range(len(groceries_list)) for tau in range(t)
        )
    )

# enforce that u matches what we're using
# for ik, R_i in enumerate(recipes_transpose_csr_ish):
for j, jv in enumerate(recipes_transpose_csr_ish[0]):
    masses_required = recipes_transpose_csr_ish[1][j]
    print(f"ingredient {j}/{len(recipes_transpose_csr_ish[0])}")
    for t in range(t_max):
        # print(masses_required)
        model.addConstrs((
            r_vars[jv[ind]]*masses_required[ind] >= u_vars.sum('*', tau, t) for tau in range(max(0, t-int(groceries_dict[groceries_list[j]]["Shelf Life (Days)"])), t) for ind in range(len(jv))
        ))


model.update()

ingredient 0/49
ingredient 1/49
ingredient 2/49
ingredient 3/49
ingredient 4/49
ingredient 5/49
ingredient 6/49
ingredient 7/49
ingredient 8/49
ingredient 9/49
ingredient 10/49
ingredient 11/49
ingredient 12/49
ingredient 13/49
ingredient 14/49
ingredient 15/49
ingredient 16/49
ingredient 17/49
ingredient 18/49
ingredient 19/49
ingredient 20/49
ingredient 21/49
ingredient 22/49
ingredient 23/49
ingredient 24/49
ingredient 25/49
ingredient 26/49
ingredient 27/49
ingredient 28/49
ingredient 29/49
ingredient 30/49
ingredient 31/49
ingredient 32/49
ingredient 33/49
ingredient 34/49
ingredient 35/49
ingredient 36/49
ingredient 37/49
ingredient 38/49
ingredient 39/49
ingredient 40/49
ingredient 41/49
ingredient 42/49
ingredient 43/49
ingredient 44/49
ingredient 45/49
ingredient 46/49
ingredient 47/49
ingredient 48/49


In [53]:
g_vars*np.array(c_list)

TypeError: unsupported operand type(s) for *: 'tupledict' and 'float'

In [56]:
g_vars.prod({k: v for k,v in enumerate(c_list)})

<gurobi.LinExpr: 0.0>

In [ ]:
model.setObjective(sum(g_vars.prod({k: v for k,v in enumerate(c_list)}) + 10*)

In [27]:
recipes_dict

{'Zurich-Style Meat Saute': {'Beef': 600.0,
  'Mushrooms': 250.0,
  'Onion': 150.0,
  'Flour': 8.0,
  'Cream': 200.0,
  'Parsley': 5.0},
 'Galinha Caipira': {'Onion': 150.0,
  'Chicken': 2500.0,
  'Chives': 50.0,
  'Butter': 15.0},
 'Grilled Mackerel with Miso Soup and Squash': {'Onion': 50.0,
  'Butter': 14.0,
  'Mackerel': 200.0,
  'Miso Paste': 30.0,
  'Squash': 150.0,
  'Rice': 185.0,
  'Soy Sauce': 15.0},
 'Risengrød': {'Butter': 50.0,
  'Rice': 180.0,
  'Milk': 1000.0,
  'Salt': 1.0,
  'Sugar': 50.0,
  'Cinnamon': 4.0},
 'Spiced Apple Pancakes': {'Flour': 250.0,
  'Milk': 160.0,
  'Sugar': 12.5,
  'Cinnamon': 8.0,
  'Apple': 200.0,
  'Egg': 50.0},
 'Tajine Maadnous': {'Onion': 150.0,
  'Parsley': 50.0,
  'Chicken': 454.0,
  'Salt': 6.0,
  'Cinnamon': 0.5,
  'Egg': 60.0,
  'Cheese': 170.0,
  'Olive Oil': 30.0},
 'Grostoli': {'Flour': 800.0,
  'Butter': 42.0,
  'Milk': 240.0,
  'Salt': 1.0,
  'Sugar': 200.0,
  'Egg': 200.0,
  'Baking Powder': 12.0},
 'Exotic Ginger Cumin Chicken': 